### 📋 Synthetic Taxpayer Dataset Schema

| Column                  | Type       | Description                                                  |
|-------------------------|------------|--------------------------------------------------------------|
| `filing_status`         | categorical| Filing status: `"single"`, `"married_joint"`, `"head_of_household"` |
| `num_dependents`        | integer    | Total number of dependents claimed                           |
| `num_children_under_17` | integer    | Number of children under age 17 (for Child Tax Credit)       |
| `child_ages`            | list[int]  | Ages of all dependent children                               |
| `investment_income`     | float      | Income from investments (affects EITC eligibility)           |
| `disability`            | boolean    | Disability status (used in some eligibility rules)           |
| `claimed_eitc`          | boolean    | Label: `True` if eligible for Earned Income Tax Credit       |
| `claimed_ctc`           | boolean    | Label: `True` if eligible for Child Tax Credit               |
| `wages`                 | float      | Earned income from work (includes distribution up to ~$1M)   |
| `agi`                   | float      | Adjusted Gross Income, clipped between $0 and $1,000,000     |



### Install Dependencies

In [89]:
import pandas as pd
import numpy as np

### Helper functions

In [90]:
# Helper Functions
def is_eligible_for_eitc(row):
    return (
        row["agi"] < 60000 and
        row["investment_income"] < 3700 and
        row["num_children_under_17"] >= 1
    )

def is_eligible_for_ctc(row):
    agi_limit = np.random.randint(low=60000, high=200000) + np.random.normal(loc=0, scale=135130)  # ±$10k jitter
    return row["agi"] < agi_limit and row["num_children_under_17"] > 0 and row["investment_income"] < 3700


### Prep Simulated Data

In [91]:
np.random.seed(421)
n = 10000

filing_statuses = np.random.choice(
    ["single", "married_joint", "head_of_household"],
    size=n,
    p=[0.4, 0.3, 0.3]
)

# Dependents
num_dependents = np.random.poisson(lam=1.5, size=n)
num_children_under_17 = np.minimum(num_dependents, np.random.poisson(lam=1.15, size=n))

# Wages
wages = np.concatenate([
    np.random.normal(loc=35000, scale=15000, size=int(n * 0.65)),
    np.random.normal(loc=150000, scale=30000, size=int(n * 0.20)),
    np.random.normal(loc=350000, scale=100000, size=int(n * 0.15))
])
wages = np.clip(wages, 0, 1_000_000)

# Investment components
interest_income = np.random.exponential(scale=300, size=n)
capital_gains = np.random.normal(loc=5000, scale=17230, size=n).clip(0)
investment_income = interest_income + capital_gains

# AGI with added noise
agi = wages + investment_income - np.random.normal(loc=3000, scale=2000, size=n)
agi += np.random.normal(loc=0, scale=1000, size=n)  # AGI noise
agi = np.clip(agi, 0, 1_000_000)

# Add disability flag
disability = np.random.choice([0, 1], size=n, p=[0.9, 0.1])

# Create DataFrame
df = pd.DataFrame({
    "filing_status": filing_statuses,
    "num_dependents": num_dependents,
    "num_children_under_17": num_children_under_17,
    "wages": wages,
    "agi": agi,
    "investment_income": investment_income,
    "interest_income": interest_income,
    "capital_gains": capital_gains,
    "disability": disability
})

### Apply eligibility logic

In [92]:
df["claimed_eitc"] = df.apply(is_eligible_for_eitc, axis=1)
df["claimed_ctc"] = df.apply(is_eligible_for_ctc, axis=1)

### Add noise to labels: flip 2% of EITC and CTC eligibility to simulate misfilings or edge cases

In [93]:
for label in ["claimed_eitc", "claimed_ctc"]:
    flip_idx = df.sample(frac=0.03, random_state=(42 if label == "claimed_eitc" else 1337)).index
    df.loc[flip_idx, label] = ~df.loc[flip_idx, label]

mask = df["filing_status"] == "head_of_household"
df.loc[mask, "num_dependents"] += np.random.binomial(n=1, p=0.4, size=mask.sum())
df.loc[mask, "agi"] -= np.random.normal(loc=5000, scale=2000, size=mask.sum())

### More dimensions

In [94]:
# Noise - more dimensions!
n = df.shape[0]
# Simulate filing age (18 to 80)
df["filing_age"] = np.random.normal(loc=38, scale=12, size=n).clip(18, 80).astype(int)
# State code (simulate 5 popular states)
df["state_code"] = np.random.choice(["CA", "TX", "NY", "FL", "IL"], size=n, p=[0.3, 0.25, 0.2, 0.15, 0.1])
# Number of income sources (W2, 1099s)
df["num_income_sources"] = np.random.poisson(lam=1.2, size=n)
# Housing status
df["housing_status"] = np.random.choice(["Own", "Rent", "Other"], size=n, p=[0.4, 0.5, 0.1])

df["wages_to_dependents"] = df["wages"] / (df["num_dependents"] + 1)
df["agi_minus_investment"] = df["agi"] - df["investment_income"]

In [95]:
    "single": 0,
    "married_joint": 1,
    "head_of_household": 2
})

In [96]:
df

filing_status  num_dependents  num_children_under_17          wages  \
0         married_joint               2                      2   32249.838835   
1     head_of_household               2                      0   46073.111946   
2                single               1                      1   33204.153424   
3     head_of_household               1                      0   33417.937273   
4                single               3                      3   35511.035551   
...                 ...             ...                    ...            ...   
9995             single               1                      0  422649.612944   
9996             single               3                      1  394032.592619   
9997      married_joint               1                      1  413201.143481   
9998  head_of_household               1                      1  307841.951319   
9999             single               0                      0  365599.985881   

                agi  investment_income  interest_income  capital_gains  \
0      29532.668884         431.527688       431.527688       0.000000   
1      51492.635036       13342.109305       580.336076   12761.773229   
2      29938.251133         467.515932       467.515932       0.000000   
3      29111.077640        7013.298018       149.500477    6863.797541   
4      31923.148427         380.898207       380.898207       0.000000   
...             ...                ...              ...            ...   
9995  416171.804148         543.829963       543.829963       0.000000   
9996  390908.474925         760.862824       760.862824       0.000000   
9997  451383.104478       36529.545091       264.437619   36265.107472   
9998  337587.809769       35735.728399       399.064142   35336.664257   
9999  375635.260385       12196.565201        44.495789   12152.069412   

      disability  claimed_eitc  claimed_ctc  filing_age state_code  \
0              0          True         True          34         TX   
1              0         False        False          43         CA   
2              0          True        False          45         FL   
3              0         False        False          20         CA   
4              0          True         True          26         CA   
...          ...           ...          ...         ...        ...   
9995           0         False        False          31         CA   
9996           0         False        False          47         TX   
9997           0         False        False          18         NY   
9998           0         False        False          21         TX   
9999           0         False        False          58         TX   

      num_income_sources housing_status  wages_to_dependents  \
0                      3            Own         10749.946278   
1                      1            Own         15357.703982   
2                      1            Own         16602.076712   
3                      0           Rent         16708.968637   
4                      0           Rent          8877.758888   
...                  ...            ...                  ...   
9995                   1           Rent        211324.806472   
9996                   3           Rent         98508.148155   
9997                   1           Rent        206600.571741   
9998                   1            Own        153920.975660   
9999                   1            Own        365599.985881   

      agi_minus_investment  filing_status_code  
0             29101.141196                   1  
1             38150.525731                   2  
2             29470.735200                   0  
3             22097.779622                   2  
4             31542.250220                   0  
...                    ...                 ...  
9995         415627.974185                   0  
9996         390147.612101                   0  
9997         414853.559386                   1  
9998         301852.081370                   2  